In [ ]:
import logging
from pathlib import Path

import cupy_xarray  # noqa: F401
import hvplot.xarray
import rioxarray  # noqa: F401
from darts_acquisition.planet import load_planet_scene
from dask.distributed import Client
from odc.stac import configure_rio
from rich import traceback
from rich.logging import RichHandler

from darts.utils.logging import LoggingManager

LoggingManager.setup_logging()
logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler(rich_tracebacks=True)],
)
traceback.install(show_locals=True)
client = Client()
configure_rio(cloud_defaults=True, aws={"aws_unsigned": True}, client=client)
client

In [ ]:
DATA_ROOT = Path("../data")

# fpath = DATA_ROOT / "input/planet/PSOrthoTile/4372514/5790392_4372514_2022-07-16_2459"
fpath = DATA_ROOT / "input/planet/PSOrthoTile/4974017/5854937_4974017_2022-08-14_2475"
arcticdem_dir = DATA_ROOT / "download/arcticdem"

In [ ]:
# load planet scene
ds_planet = load_planet_scene(fpath)  # .isel(x=slice(0, 2000), y=slice(6000, 8000))
ds_planet

## Get the Tile

In [ ]:
from darts_acquisition.arcticdem import load_arcticdem_tile

ds = load_arcticdem_tile(ds_planet.odc.geobox, arcticdem_dir, resolution=2, buffer=0, persist=True)
ds

In [ ]:
crs = ds.rio.crs.to_string()
ds.dem.hvplot.image(aggregator="max", rasterize=True, cmap="terrain", crs=crs, projection=crs)

In [ ]:
crs = ds_planet.rio.crs.to_string()
dem_plot = ds.dem.rio.reproject_match(ds_planet).hvplot.image(
    aggregator="max", rasterize=True, cmap="terrain", data_aspect=1, crs=crs, projection=crs
)
red_plot = ds_planet.red.hvplot.image(
    x="x", y="y", aggregator="mean", rasterize=True, cmap="reds", data_aspect=1, crs=crs, projection=crs
)
dem_plot + red_plot

## Calculate Relative Elevation and Slope

### Numpy

Topographic Position Index calculated in 652.77 seconds.
Slope calculated in 3.67 seconds.

### Dask (4 worker)

Topographic Position Index calculated in 135.34 seconds.
Slope calculated in 4.33 seconds.

### Cupy

Topographic Position Index calculated in 12.69 seconds. 
Slope calculated in 0.16 seconds.


In [ ]:
from darts_preprocessing.engineering.arcticdem import calculate_slope, calculate_topographic_position_index
from xrspatial.utils import has_cuda_and_cupy

use_cupy = True
if use_cupy and has_cuda_and_cupy():
    ds = ds.cupy.as_cupy()
ds = calculate_topographic_position_index(ds)
ds = calculate_slope(ds)
if ds.cupy.is_cupy:
    ds = ds.cupy.as_numpy()
ds

In [ ]:
crs = ds.rio.crs.to_string()
dem_plot = ds.dem.hvplot.image(
    x="x", y="y", aggregator="max", rasterize=True, cmap="terrain", data_aspect=1, crs=crs, projection=crs, title="DEM"
)
tpi_plot = ds.tpi.hvplot.image(
    x="x", y="y", aggregator="max", rasterize=True, cmap="terrain", data_aspect=1, crs=crs, projection=crs, title="TPI"
)
slope_plot = ds.slope.hvplot.image(
    x="x", y="y", aggregator="max", rasterize=True, data_aspect=1, crs=crs, projection=crs, title="Slope"
)
dem_plot + tpi_plot + slope_plot